In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

> # Read the data

In [ ]:
oil_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
holidays_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
stores_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
train_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
transactions_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

> **train and test data frame are the same features except sales in train data**

In [ ]:
oil_df.head()

In [ ]:
holidays_df.head()

In [ ]:
stores_df.head()

In [ ]:
transactions_df.head()

 > **Wrangle the train data frame**

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.duplicated().sum()

In [ ]:
round(train_df.describe(), 2)

> # EDA

In [ ]:
train_df.id.nunique()

In [ ]:
round(train_df.sales.describe(), 2)

In [ ]:
train_df.info()

In [ ]:
train_df.date = pd.to_datetime(train_df.date)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(train_df.date, train_df.sales)
plt.show()

> **Group Sales by month**

In [ ]:
months_sales = train_df.groupby(train_df['date'].dt.strftime('%B'))['sales'].sum().sort_values()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(months_sales)
plt.title("the sales by month", fontsize=15)
plt.xlabel("the month", fontsize=13)
plt.ylabel("the sales", fontsize=13)
plt.show()

> **Sales Per year**

In [ ]:
plt.figure(figsize=(15,8))
train_df.groupby(pd.Grouper(key='date', freq='2D')).sum()['sales'].plot()
plt.ylabel("the sales", fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
train_df.groupby(pd.Grouper(key='date', freq='1Y')).sum()['sales'].plot()
plt.title("the sales per year", fontsize=15)
plt.ylabel("the sales", fontsize=15)
plt.show()

> **Sales decreased after 2016**

In [ ]:
train_df.head(2)

> **What's the most sales family**

In [ ]:
plt.figure(figsize=(15, 8))
train_df.groupby("family")['family'].count().plot(kind='bar')
plt.title("the count of each family sales", fontsize=15)
plt.show()

> ALL has the same count

**Group on-pormotion by month**

In [ ]:
months_pormotions = train_df.groupby(train_df['date'].dt.strftime('%B'))['onpromotion'].sum().sort_values()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(months_pormotions)
plt.title("the pormotions by month", fontsize=15)

plt.xlabel("the month", fontsize=13)
plt.ylabel("the pormotions", fontsize=13)
plt.show()

**Group pormotions by year**

In [ ]:
plt.figure(figsize=(10,8))
train_df.groupby(pd.Grouper(key='date', freq='1Y')).sum()['onpromotion'].plot()
plt.title("the promotion per year", fontsize=15)
plt.ylabel("the promotions", fontsize=15)
plt.show()

> **The sales per store**

In [ ]:
stores_sales = train_df.groupby('store_nbr')['sales'].sum()

In [ ]:
stores_sales.sort_values(inplace=True)

In [ ]:
round(stores_sales, 2)

> **The top 10 sales per store**

In [ ]:
plt.figure(figsize=(10, 8))
stores_sales[-10:].plot(kind='bar')
plt.title("The top 10 sales per store", fontsize=15)
plt.xlabel("the store")
plt.ylabel("the sales")
plt.show()

>**Stores data frame EDA** 

In [ ]:
stores_df.info()

In [ ]:
stores_df.head(2)

In [ ]:
sales_nums = stores_sales.index

In [ ]:
top_stores = sales_nums[-10:]

In [ ]:
cities = []
i = 0
for city in stores_df.city:
    if stores_df.store_nbr[i] in  top_stores:
        cities.append(stores_df.city[i])
    i += 1

In [ ]:
cities

In [ ]:
cities = np.array(cities)
cities = pd.Series(cities)

> **The top sales per city**

In [ ]:
cities.hist()
plt.title("the freq of top sales stores per city")
plt.ylabel("the count of top stores on the city")
plt.show()

> #   Modeling

> **DecisionTreeRegressor**

In [ ]:
train_df.date = pd.to_numeric(train_df.date)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
train_df.family = le.fit_transform(train_df.family)

In [ ]:
train_df.head(2)

In [ ]:
X = train_df.drop(['sales', 'id', 'date'], axis = 1).values
y = train_df.sales.values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
regressor = DecisionTreeRegressor(max_depth=5, max_leaf_nodes=12, random_state=42)

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
regressor.score(X_train, y_train)

In [ ]:
regressor.score(X_test, y_test)

> **Linear Regression**

> Score: 59

In [ ]:
plt.scatter(train_df.onpromotion, train_df.sales)

In [ ]:
train_df.corr()

In [ ]:
# from sklearn.linear_model import LinearRegression

In [ ]:
# regressor = LinearRegression()

In [ ]:
# train_df.iloc[:,-1:]

In [ ]:
# X = train_df.iloc[:,-1:].values
# y = train_df.sales.values

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
# regressor.fit(X_train, y_train)

In [ ]:
# regressor.score(X_test, y_test)

> **XGBoost Regressor**

In [ ]:
import xgboost as xg

In [ ]:
xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 123, max_depth=10)

In [ ]:
xgb_r.fit(X_train, y_train)

In [ ]:
xgb_r.score(X_test, y_test)

> # Predictions 

In [ ]:
test_df.head(2)

In [ ]:
test_df.family = le.fit_transform(test_df.family)

In [ ]:
test_df.head(2)

In [ ]:
X = test_df.drop(["id", "date"], axis=1).values
X.shape

In [ ]:
X[0]

> DS tree regressor

In [ ]:
# predictions = regressor.predict(X)

> XGBosst Regressor

In [ ]:
predictions = xgb_r.predict(X)

In [ ]:
test_df['sales'] = predictions

In [ ]:
test_df.head(2)

In [ ]:
test_df.drop(['store_nbr', 'family', 'onpromotion', 'date'], axis=1, inplace=True)

In [ ]:
test_df.head(2)

In [ ]:
test_df.to_csv('submission.csv', index=False)
print("submission successed")